In [325]:
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.nn.functional as F
# torhc dataset
from torch.utils.data import Dataset, DataLoader

# train test split
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import ReduceLROnPlateau

import pandas as pd
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from torchviz import make_dot

#import summary for models 
from torchsummary import summary

import numpy as np

In [326]:
train_test: pd.DataFrame = pd.read_csv('../Data/train_validate.csv')

# # Apply SMOTE to the training set
# smote = SMOTE(random_state=42)
# X_train, y_train = smote.fit_resample(X_train, y_train)

# Make a pytorch dataset
class EEGDS(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)  
        self.y = torch.tensor(y, dtype=torch.float32)   

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train, test = train_test_split(train_test, test_size=0.2, random_state=42)
train: pd.DataFrame
X_train = train.drop(columns=train.filter(regex='main.*'))
y_train = train.filter(regex='main.*')
X_validate = test.drop(columns=test.filter(regex='main.*'))
y_validate = test.filter(regex='main.*')

# Apply pca to the training set
pca = PCA(n_components=300)
X_train = pca.fit_transform(X_train)
X_validate = pca.transform(X_validate)
    
train_dataset = EEGDS(X_train, y_train.to_numpy())
validate_dataset = EEGDS(X_validate, y_validate.to_numpy())

# data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
validate_loader = DataLoader(validate_dataset, batch_size=32, shuffle=False)

In [327]:
class EEGClassifier(nn.Module):
    def __init__(self, input_dim, output_dim, neurons, dropout=0.10):
        super(EEGClassifier, self).__init__()
        
        self.layer1 = nn.Linear(input_dim, neurons[0])
        self.layer2 = nn.Linear(neurons[0], neurons[1])
        self.layer3 = nn.Linear(neurons[1], neurons[2])
        self.layer4 = nn.Linear(neurons[2], neurons[3])
        self.layer5 = nn.Linear(neurons[3], neurons[4])
        self.output_layer = nn.Linear(neurons[4], output_dim)

        self.bn1 = nn.BatchNorm1d(neurons[0])
        self.bn2 = nn.BatchNorm1d(neurons[1])
        self.bn3 = nn.BatchNorm1d(neurons[2])
        self.bn4 = nn.BatchNorm1d(neurons[3])
        self.bn5 = nn.BatchNorm1d(neurons[4])

        self.gelu = nn.GELU()

        self.dropout = nn.Dropout(p=dropout)
        
        init.xavier_uniform_(self.layer1.weight)
        init.xavier_uniform_(self.layer2.weight)
        init.xavier_uniform_(self.layer3.weight)
        init.xavier_uniform_(self.layer4.weight)
        init.xavier_uniform_(self.layer5.weight)
        init.xavier_uniform_(self.output_layer.weight)

    def forward(self, x):
        x = self.gelu(self.bn1(self.layer1(x))) 
        x = self.dropout(x)
        
        x = self.gelu(self.bn2(self.layer2(x)))
        x = self.dropout(x)
        
        x = self.gelu(self.bn3(self.layer3(x)))
        x = self.dropout(x)
        
        x = self.gelu(self.bn4(self.layer4(x)))
        x = self.dropout(x)
        
        x = self.gelu(self.bn5(self.layer5(x)))
        
        x = self.output_layer(x)
        
        return x

In [328]:
model = EEGClassifier(300, 7, [512, 256, 128, 64, 32], 0.1)
model.eval()
sample_input = torch.randn(1, 300)  

output = model(sample_input)
output

tensor([[ 0.1985, -0.0298, -0.0870, -0.0279,  0.0346,  0.0324, -0.1007]],
       grad_fn=<AddmmBackward0>)

In [329]:
summary(model, input_size=(300, ))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 512]         154,112
       BatchNorm1d-2                  [-1, 512]           1,024
              GELU-3                  [-1, 512]               0
           Dropout-4                  [-1, 512]               0
            Linear-5                  [-1, 256]         131,328
       BatchNorm1d-6                  [-1, 256]             512
              GELU-7                  [-1, 256]               0
           Dropout-8                  [-1, 256]               0
            Linear-9                  [-1, 128]          32,896
      BatchNorm1d-10                  [-1, 128]             256
             GELU-11                  [-1, 128]               0
          Dropout-12                  [-1, 128]               0
           Linear-13                   [-1, 64]           8,256
      BatchNorm1d-14                   

In [330]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), 
                       lr=0.0001,
                       weight_decay=1e-5)

scheduler = ReduceLROnPlateau(optimizer, 
                              mode='min', 
                              factor = 0.1, 
                              patience = 3, 
                              min_lr = 1e-5, 
                              verbose = True)

/Users/hamsterlord/.pyenv/versions/3.10.4/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [331]:
for data in train_loader:
    inputs, labels = data
    print(inputs.shape)
    print(labels.shape)
    break

torch.Size([32, 300])
torch.Size([32, 7])


In [ ]:
best_loss = np.inf
patience = 20
min_delta = 0.01
early_stop_counter = 0

train_losses = []
val_losses = []
val_accs = []
final_epoch = 0

num_epochs = 100

for epoch in range(num_epochs):
    
    # train
    model.train()
    running_loss = 0.
    running_accuracy = 0.

    for data in train_loader:
        inputs, labels = data
        
        optimizer.zero_grad()  # zero gradients
        
        outputs = model.forward(inputs)  # input net
        
        #print(type(outputs))
        
        labels = labels.argmax(dim=1)
        labels = labels.long()

        loss = criterion(outputs, labels)
        loss.backward()

#         outputs = F.softmax(outputs, dim=1)
#         top_p, top_class = outputs.topk(k=1, dim=1)

#         equals = top_class == labels.view(*top_class.shape)

#         running_accuracy += torch.mean(equals.type(torch.FloatTensor))

        optimizer.step()

        running_loss += loss.item()
        
#         _, predicted = torch.max(outputs.data, 1)  
#         accuracy = (predicted == labels).float().mean()
#         running_accuracy += accuracy.item()

    epoch_loss = running_loss / len(train_loader)
    #epoch_accuracy = running_accuracy / len(train_loader)
    
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}")
    train_losses.append(epoch_loss)
    
    # val
    model.eval()
    val_loss = 0.0
    val_accuracy = 0.0
    correct = total = 0

    with torch.no_grad():
        for data in validate_loader:
            inputs, labels = data

            outputs = model(inputs)
            labels = labels.float()

            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            _, labels = torch.max(labels.data, 1)
            
            accuracy = (predicted == labels).float().mean()
            val_accuracy += accuracy.item()

    val_loss /= len(validate_loader)
    val_accuracy /= len(validate_loader)

    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}\n")
    val_losses.append(val_loss)
    val_accs.append(val_accuracy)

    
    # Early stopping
    if val_loss < best_loss - min_delta:
        best_loss = val_loss
        early_stop_counter = 0  
        #best_model_state = model.state_dict()  
    else:
        early_stop_counter += 1
    
    # if early_stop_counter >= patience:
    #     print(f"Early stopping at epoch {epoch+1}.")
    #     final_epoch = epoch+1
    #     #model.load_state_dict(best_model_state) 
    #     break
        
    scheduler.step(val_loss)

Epoch [1/100], Loss: 2.2360
Validation Loss: 2.0920, Validation Accuracy: 0.1293

Epoch [2/100], Loss: 2.1674
Validation Loss: 2.0733, Validation Accuracy: 0.1709

Epoch [3/100], Loss: 2.1252
Validation Loss: 2.0538, Validation Accuracy: 0.1657

